# Response spectra

The response spectra of a given earthquake describe the peak response of all possible SDOF systems.
The peaks of the displacement for different $\omega$ and $\xi$ values is known as the displacement spectrum:
$$
S_u(\omega,\xi) = \max_t\{u(t)\}
$$

Similarly, the pseudo velocity and pseudo acceleration spectra are:
\begin{align}
S_v(\omega,\xi) &= \omega S_u(\omega,\xi) \approx \max_t\{\dot{u}(t)\} \\
S_a(\omega,\xi) &= \omega^2 S_u(\omega,\xi) \approx \max_t\{\ddot{u}(t) + a_g\}
\end{align}

Given a value of $\omega$ and $\xi$, the corresponding response value can be obtained integrating the equation of motion and finding the max. This procedure can ber repeated as many times needed to generate the response spectra.

Loading useful python libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Reading the seismic data.

In [ ]:
g = 9.81            # m/s^2
dt = 0.02           # the time step considered by the input data
seism_scale = 0.01  # convert dm/s^2 to m/s^2
seism = np.loadtxt('AcMx1985.txt') * seism_scale
times = np.arange(len(seism))*dt

plt.plot(times, seism)
plt.show()

Definition of the values of the possible structures. As the number of sampling periods increase, the definition of the spectra will be more accurate, but the computations will be slower.

In [ ]:
damping = 0.05
periods = np.linspace(start=0.01,stop=2,num=50)

## Duhamel method

In this section, the equation of motion is solved using the Duhamel convolution. For each period of the structures, the Duhamel integral is performed.

In [ ]:
def FreeVibration(t, frequency, damping):
    return 1/frequency * np.exp(-damping*frequency*t) * np.sin(frequency*t)

def MaxResponseDuhamel(period, damping):
    frequency = 2*np.pi/period
    free_response = FreeVibration(times, frequency, damping)
    u_duhamel = np.convolve(seism, free_response, 'full') * dt
    u_duhamel = u_duhamel[:len(u_duhamel)//2+1]
    a_duhamel = frequency**2*u_duhamel
    return max(np.absolute(a_duhamel))

Sa_duhamel = [MaxResponseDuhamel(T,damping) for T in periods]

## Direct integration

In this section, the differential equation is integrated using a numerical scheme.

We are using a standard python library. The main drawback of the library is that needs to interpolate the seismic data. Therefore, this method is slower, although more accurate. A custom implementation will be faster while keeping the same level of accuracy.

In [ ]:
from scipy.integrate import solve_ivp

def SystemDerivatives(t, y, frequency, damping):
    u = y[0]
    v = y[1]
    s = np.interp(t, times, seism)
    a = s - 2*damping*frequency*v - frequency**2*u
    return [v, a]

def MaxResponseInt(period, damping):
    frequency = 2*np.pi/period
    solution = solve_ivp(fun=SystemDerivatives, t_span=[0, times[-1]],
                         y0=[0,0], t_eval=times, args=[frequency, damping])
    a = frequency**2*solution.y[0]
    return max(np.absolute(a))

Sa_integration = [MaxResponseInt(T,damping) for T in periods]

## Comparison of the results

In general, both methods are in good agreement. However, for small periods, there is a difference from the Duhamel and the direct integration methods. Therefore, it's recommended to use the spectra obtained from the direct integration.

In [ ]:
plt.plot(periods,Sa_duhamel,label='Duhamel')
plt.plot(periods,Sa_integration,label='integration')
plt.title(f'Accelerogram spectrum, damping {damping:.0%}')
plt.ylabel('Sa/g')
plt.xlabel('T [s]')
plt.legend()
plt.show()

In [ ]:
Sa = Sa_integration